<a href="https://colab.research.google.com/github/querzy/hello-world/blob/master/Model_2_LSTM_PREDICTIONS_MULTIPARITE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

REINITIALISER L'ENVIRONNEMENT D'EXECUTION !!!
- Execution - reinitialiser l'environnement d'execution
- Executer le script plusieurs fois
- Utiliser le dossier Oanda currency download2 (4000 barres)
- Retirer une barre du fichier si barre ouverte et considérer le resultat à t0 OU considérer le résultat à t-1 avec une barre ouverte.

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving BTCUSD1440.csv to BTCUSD1440.csv
Saving DOLLAR_INDX1440.csv to DOLLAR_INDX1440.csv
Saving EURUSD1440.csv to EURUSD1440.csv


In [ ]:
#@title
import numpy as np
import pandas as pd
from datetime import datetime
from pandas.tseries.offsets import DateOffset
from sklearn.preprocessing import MinMaxScaler
 
import tensorflow as tf
from tensorflow import keras
 
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout
import warnings
warnings.filterwarnings("ignore")
#import chart_studio.plotly as py
#import plotly.plotly as py
import plotly.offline as pyoff
import plotly.graph_objs as go
#pyoff.init_notebook_mode(connected=True)
from numpy.random import seed
seed(1)
tf.random.set_seed(2) 
 
 
#def parser(x):
#    return pd.datetime.strptime('190'+x, '%Y-%m')
 
 
#start='2020.08.21'
#rng=pd.date_range(start, periods=5, freq='B').strftime('%Y-%m-%d 21:00:00')
#df = pd.DataFrame({ 'date': rng, 'qty': np.random.randn(len(rng)) }) 
 
#plot daily prices
#plot_data = [
#    go.Scatter(
#        x=df['date'],
#        y=df['qty'],
#    )
#]
 
#plot_layout = go.Layout(
#        title='Daily Price'
#    )
#fig = go.Figure(data=plot_data, layout=plot_layout)
#fig.show()
 
def mean(numbers):
    return float(sum(numbers)) / max(len(numbers), 1)
 
def get_num(xx):
    return int(''.join(ele for ele in xx if ele.isdigit()))


with open('/content/recap deep learning.txt', 'w') as document: pass

fileLst=['EURUSD1440.csv','DOLLAR_INDX1440.csv','BTCUSD1440.csv'] 
  
for k in fileLst :
    filename1 = datetime.now().strftime("%d %b %Y - %H%M")
    filename=k.replace('.csv','')
    filename= filename+' '+filename1
    valperiod=get_num(k)
    print(valperiod)
    if valperiod==10080 :
        TimeFrame = 'Weekly'
        epochs=int(6)
    elif valperiod==1440 :
        TimeFrame = 'Daily'
        epochs=int(5)
    elif valperiod==240 :
        TimeFrame = '4H'
        epochs=int(3)
    elif valperiod==60 :
        TimeFrame = '1H'
        epochs=int(6)
    elif valperiod==15 :
        TimeFrame = '15 min'
        epochs=int(5)
 
    t0up=0
    t0dn=0 
    PRICEAVt0=[]
    PRICEAVupt0=[]
    PRICEAVdnt0=[]
    for j in range(2):
      df = pd.read_csv(k, index_col='Date',usecols=['Date', 'Close'],date_parser = pd.to_datetime)
      df.tail()
     
      print(df)
     
      train = df
     
      scaler = MinMaxScaler()
      scaler.fit(train)
      train = scaler.transform(train)
     
      n_input = 12
      n_features = 1
      generator = TimeseriesGenerator(train, train, length=n_input, batch_size=6)
     
      model = Sequential()
      #model.add(LSTM(200, activation='relu', input_shape=(n_input, n_features)))
      model.add(LSTM(100, activation='relu', input_shape=(n_input, n_features)))
      #model.add(Dropout(0.15))
      model.add(Dense(1))
     
      #Model summary
      #model.summary()
      #Compiling
      #model.compile(optimizer='adam', loss = 'mse')
     
      optimizer = keras.optimizers.Adam(learning_rate=0.001)
      model.compile(optimizer=optimizer, loss='mse')
     
      history = model.fit(generator,epochs=5,verbose=1)#EURUSD
      #history = model.fit(generator,epochs=20,verbose=1)#BTCUSD
     
      hist = pd.DataFrame(history.history)
      hist['epoch'] = history.epoch
     
      plot_data = [
          go.Scatter(
              x=hist['epoch'],
              y=hist['loss'],
              name='loss'
          )
          
      ]
     
      plot_layout = go.Layout(
              title='Training loss'
          )
      fig = go.Figure(data=plot_data, layout=plot_layout)
      #fig.show()
      pyoff.iplot(fig)
     
      pred_list = []
     
      batch = train[-n_input:].reshape((1, n_input, n_features))
     
      for i in range(n_input):   
          pred_list.append(model.predict(batch)[0]) 
          batch = np.append(batch[:,1:,:],[[pred_list[i]]],axis=1)
     
      if valperiod ==10080 :
        add_dates = [df.index[-1] + DateOffset(weeks=x) for x in range(0,13) ]
      elif valperiod ==1440 :
        add_dates = [df.index[-1] + DateOffset(days=x) for x in range(0,13) ]
      elif valperiod ==240 :
        add_dates = [df.index[-1] + DateOffset(hours=x*4) for x in range(0,13) ]  
      elif valperiod ==60 :
        add_dates = [df.index[-1] + DateOffset(hours=x) for x in range(0,13) ] 
      #add_dates = [df.index[-1] + DateOffset(weeks=x) for x in range(0,13) ]
      #add_dates = [df.index[-1] + DateOffset(days=x) for x in range(0,13) ]
      #add_dates = [df.index[-1] + DateOffset(hours=x*4) for x in range(0,13) ]
      
      future_dates = pd.DataFrame(index=add_dates[1:],columns=df.columns)
      print(add_dates)
      df_predict = pd.DataFrame(scaler.inverse_transform(pred_list),
                                index=future_dates[-n_input:].index, columns=['Prediction'])
     
      df_proj = pd.concat([df,df_predict], axis=1)
     
      df_proj.tail(12)
     
      plot_data = [
          go.Scatter(
              x=df_proj.index[-30:],
              y=df_proj['Close'][-30:],
              name='actual'
          ),
          go.Scatter(
              x=df_proj.index[-30:],
              y=df_proj['Prediction'][-30:],
              name='prediction'
          )
      ]
     
      plot_layout = go.Layout(
              title=k+' prediction t0'
          )
     
      fig = go.Figure(data=plot_data, layout=plot_layout)
      #fig.show()
      pyoff.iplot(fig)
      
      
      yt0Cl=df['Close'].iloc[-1]
      yt0Pr=df_proj['Prediction'].iloc[-12]
      PRICEAVt0.append(yt0Pr)
      print(yt0Cl)
      print(yt0Pr)
      t0diff=yt0Cl-yt0Pr
      if t0diff>0:
        t0dn=t0dn+1
        PRICEAVdnt0.append(yt0Pr)
      elif t0diff<=0 :
        t0up=t0up+1
        PRICEAVupt0.append(yt0Pr)
          
    prixmoyt0=mean(PRICEAVt0)
    prixmoydnt0=mean(PRICEAVdnt0)
    prixmoyupt0=mean(PRICEAVupt0)
      #########################################
      #########################################
     
    t1up=0
    t1dn=0 
    PRICEAVt1=[]
    PRICEAVupt1=[]
    PRICEAVdnt1=[]
    for j in range(20):
      df = pd.read_csv(k, skipfooter=1, engine='python', index_col='Date',usecols=['Date', 'Close'],date_parser = pd.to_datetime)
      df.tail()
     
      print(df)
     
      train = df
     
      scaler = MinMaxScaler()
      scaler.fit(train)
      train = scaler.transform(train)
     
      n_input = 12
      n_features = 1
      generator = TimeseriesGenerator(train, train, length=n_input, batch_size=6)
     
      model = Sequential()
      #model.add(LSTM(200, activation='relu', input_shape=(n_input, n_features)))
      model.add(LSTM(100, activation='relu', input_shape=(n_input, n_features)))
      #model.add(Dropout(0.15))
      model.add(Dense(1))
     
      #Model summary
      #model.summary()
      #Compiling
      #model.compile(optimizer='adam', loss = 'mse')
     
      optimizer = keras.optimizers.Adam(learning_rate=0.001)
      model.compile(optimizer=optimizer, loss='mse')
     
      history = model.fit(generator,epochs=5,verbose=1)#EURUSD
      #history = model.fit(generator,epochs=20,verbose=1)#BTCUSD
     
      hist = pd.DataFrame(history.history)
      hist['epoch'] = history.epoch
     
      plot_data = [
          go.Scatter(
              x=hist['epoch'],
              y=hist['loss'],
              name='loss'
          )
          
      ]
     
      plot_layout = go.Layout(
              title='Training loss'
          )
      fig = go.Figure(data=plot_data, layout=plot_layout)
      #fig.show()
      pyoff.iplot(fig)
     
      pred_list = []
     
      batch = train[-n_input:].reshape((1, n_input, n_features))
     
      for i in range(n_input):   
          pred_list.append(model.predict(batch)[0]) 
          batch = np.append(batch[:,1:,:],[[pred_list[i]]],axis=1)
     
      if valperiod ==10080 :
        add_dates = [df.index[-1] + DateOffset(weeks=x) for x in range(0,13) ]
      elif valperiod ==1440 :
        add_dates = [df.index[-1] + DateOffset(days=x) for x in range(0,13) ]
      elif valperiod ==240 :
        add_dates = [df.index[-1] + DateOffset(hours=x*4) for x in range(0,13) ]  
      elif valperiod ==60 :
        add_dates = [df.index[-1] + DateOffset(hours=x) for x in range(0,13) ] 
      
      #add_dates = [df.index[-1] + DateOffset(weeks=x) for x in range(0,13) ]
      #add_dates = [df.index[-1] + DateOffset(days=x) for x in range(0,13) ]
      #add_dates = [df.index[-1] + DateOffset(hours=x*4) for x in range(0,13) ]
      
      future_dates = pd.DataFrame(index=add_dates[1:],columns=df.columns)
      print(add_dates)
      df_predict = pd.DataFrame(scaler.inverse_transform(pred_list),
                                index=future_dates[-n_input:].index, columns=['Prediction'])
     
      df_proj = pd.concat([df,df_predict], axis=1)
     
      df_proj.tail(12)
     
      plot_data = [
          go.Scatter(
              x=df_proj.index[-30:],
              y=df_proj['Close'][-30:],
              name='actual'
          ),
          go.Scatter(
              x=df_proj.index[-30:],
              y=df_proj['Prediction'][-30:],
              name='prediction'
          )
      ]
     
      plot_layout = go.Layout(
              title=k+' prediction t-1'
          )
      fig = go.Figure(data=plot_data, layout=plot_layout)
      #fig.show()
      pyoff.iplot(fig)
     
      yt1Cl=df['Close'].iloc[-1]
      yt1Pr=df_proj['Prediction'].iloc[-12]
      PRICEAVt1.append(yt1Pr)
      print(yt1Cl)
      print(yt1Pr)
      t1diff=yt1Cl-yt1Pr
      if t1diff>0:
        t1dn=t1dn+1
        PRICEAVdnt1.append(yt1Pr)
      elif t1diff<=0 :
        t1up=t1up+1
        PRICEAVupt1.append(yt1Pr)
     
    prixmoyt1=mean(PRICEAVt1) 
    prixmoydnt1=mean(PRICEAVdnt1)
    prixmoyupt1=mean(PRICEAVupt1)
    print(' ')
    print('t0 :')
    print(' ')
    print('prix moyen t0 : ',prixmoyt0)
    print(' ')
    print('t0up : ',t0up)
    print('prix moyen up t0 : ',prixmoyupt0)
    print(' ')
    print('t0dn : ',t0dn)
    print('prix moyen dn t0 : ',prixmoydnt0)
    print(' ')
    print('********')
    print(' ')
    print('t1 :')
    print(' ')
    print('prix moyen t1 : ',prixmoyt1)
    print(' ')
    print('t1up : ',t1up)
    print('prix moyen up t1 : ',prixmoyupt1)
    print(' ')
    print('t1dn : ',t1dn)
    print('prix moyen dn t1 : ',prixmoydnt1)
    print(' ')

    with open('/content/recap deep learning.txt', 'a') as f:
        f.write(filename + '\n')
        f.write(' ' + '\n')
        f.write(' ' + '\n')
        f.write('t0 :'  + '\n')
        f.write(' '  + '\n')
        f.write('prix moyen t0 : '+str(prixmoyt0)  + '\n') 
        f.write(' '  + '\n')
        f.write('t0up : '+str(t0up)  + '\n')
        f.write('prix moyen up t0 : '+str(prixmoyupt0)  + '\n')
        f.write(' '  + '\n')
        f.write('t0dn : '+str(t0dn)  + '\n')
        f.write('prix moyen dn t0 : '+str(prixmoydnt0)  + '\n')
        f.write(' '  + '\n')
        f.write('********'  + '\n')
        f.write(' '  + '\n')
        f.write('t1 :'  + '\n')
        f.write(' '  + '\n')
        f.write('prix moyen t1 : '+str(prixmoyt1)  + '\n')
        f.write(' '  + '\n')
        f.write('t1up : '+str(t1up)  + '\n')
        f.write('prix moyen up t1 : '+str(prixmoyupt1)  + '\n')
        f.write(' '  + '\n')
        f.write('t1dn : '+str(t1dn)  + '\n')
        f.write('prix moyen dn t1 : '+str(prixmoydnt1)  + '\n')
        f.write(' '  + '\n')
        f.write('####################'  + '\n')
        f.write(' '  + '\n')

f.close()    

from google.colab import files
files.download('/content/recap deep learning.txt')


1440
                       Close
Date                        
2007-07-05 21:00:00  1.36278
2007-07-07 21:00:00  1.36325
2007-07-08 21:00:00  1.36267
2007-07-09 21:00:00  1.37472
2007-07-10 21:00:00  1.37455
...                      ...
2021-02-07 22:00:00  1.20513
2021-02-08 22:00:00  1.21186
2021-02-09 22:00:00  1.21186
2021-02-10 22:00:00  1.21298
2021-02-11 22:00:00  1.21282

[3999 rows x 1 columns]
Epoch 1/5
665/665 [==============================] - 5s 6ms/step - loss: 0.0191
Epoch 2/5
665/665 [==============================] - 4s 6ms/step - loss: 7.7732e-04
Epoch 3/5
665/665 [==============================] - 4s 6ms/step - loss: 5.9535e-04
Epoch 4/5
665/665 [==============================] - 4s 6ms/step - loss: 6.1663e-04
Epoch 5/5
665/665 [==============================] - 4s 6ms/step - loss: 5.2224e-04


[Timestamp('2021-02-11 22:00:00'), Timestamp('2021-02-12 22:00:00'), Timestamp('2021-02-13 22:00:00'), Timestamp('2021-02-14 22:00:00'), Timestamp('2021-02-15 22:00:00'), Timestamp('2021-02-16 22:00:00'), Timestamp('2021-02-17 22:00:00'), Timestamp('2021-02-18 22:00:00'), Timestamp('2021-02-19 22:00:00'), Timestamp('2021-02-20 22:00:00'), Timestamp('2021-02-21 22:00:00'), Timestamp('2021-02-22 22:00:00'), Timestamp('2021-02-23 22:00:00')]


1.21282
1.2115962087213996
                       Close
Date                        
2007-07-05 21:00:00  1.36278
2007-07-07 21:00:00  1.36325
2007-07-08 21:00:00  1.36267
2007-07-09 21:00:00  1.37472
2007-07-10 21:00:00  1.37455
...                      ...
2021-02-07 22:00:00  1.20513
2021-02-08 22:00:00  1.21186
2021-02-09 22:00:00  1.21186
2021-02-10 22:00:00  1.21298
2021-02-11 22:00:00  1.21282

[3999 rows x 1 columns]
Epoch 1/5
665/665 [==============================] - 5s 6ms/step - loss: 0.0094
Epoch 2/5
665/665 [==============================] - 4s 6ms/step - loss: 7.9775e-04
Epoch 3/5
665/665 [==============================] - 4s 6ms/step - loss: 8.7770e-04
Epoch 4/5
665/665 [==============================] - 4s 6ms/step - loss: 5.0651e-04
Epoch 5/5
665/665 [==============================] - 4s 6ms/step - loss: 4.7756e-04


[Timestamp('2021-02-11 22:00:00'), Timestamp('2021-02-12 22:00:00'), Timestamp('2021-02-13 22:00:00'), Timestamp('2021-02-14 22:00:00'), Timestamp('2021-02-15 22:00:00'), Timestamp('2021-02-16 22:00:00'), Timestamp('2021-02-17 22:00:00'), Timestamp('2021-02-18 22:00:00'), Timestamp('2021-02-19 22:00:00'), Timestamp('2021-02-20 22:00:00'), Timestamp('2021-02-21 22:00:00'), Timestamp('2021-02-22 22:00:00'), Timestamp('2021-02-23 22:00:00')]


1.21282
1.21151424205184
                       Close
Date                        
2007-07-05 21:00:00  1.36278
2007-07-07 21:00:00  1.36325
2007-07-08 21:00:00  1.36267
2007-07-09 21:00:00  1.37472
2007-07-10 21:00:00  1.37455
...                      ...
2021-02-04 22:00:00  1.20466
2021-02-07 22:00:00  1.20513
2021-02-08 22:00:00  1.21186
2021-02-09 22:00:00  1.21186
2021-02-10 22:00:00  1.21298

[3998 rows x 1 columns]
Epoch 1/5
665/665 [==============================] - 5s 6ms/step - loss: 0.0200
Epoch 2/5
665/665 [==============================] - 4s 6ms/step - loss: 0.0011
Epoch 3/5
665/665 [==============================] - 4s 6ms/step - loss: 8.1256e-04
Epoch 4/5
665/665 [==============================] - 4s 6ms/step - loss: 5.8203e-04
Epoch 5/5
665/665 [==============================] - 4s 6ms/step - loss: 5.8263e-04


...

47953.38
43788.228751869196
               Close
Date                
2015-02-11    215.70
2015-02-12    217.25
2015-02-13    230.25
2015-02-15    230.53
2015-02-16    233.76
...              ...
2021-02-07  38802.03
2021-02-08  46333.67
2021-02-09  46443.70
2021-02-10  44787.33
2021-02-11  47953.38

[2060 rows x 1 columns]
Epoch 1/5
342/342 [==============================] - 3s 7ms/step - loss: 0.0034
Epoch 2/5
342/342 [==============================] - 2s 7ms/step - loss: 0.0019
Epoch 3/5
342/342 [==============================] - 2s 7ms/step - loss: 4.3260e-04
Epoch 4/5
342/342 [==============================] - 2s 7ms/step - loss: 5.3000e-04
Epoch 5/5
342/342 [==============================] - 2s 7ms/step - loss: 4.0566e-04


[Timestamp('2021-02-11 00:00:00'), Timestamp('2021-02-12 00:00:00'), Timestamp('2021-02-13 00:00:00'), Timestamp('2021-02-14 00:00:00'), Timestamp('2021-02-15 00:00:00'), Timestamp('2021-02-16 00:00:00'), Timestamp('2021-02-17 00:00:00'), Timestamp('2021-02-18 00:00:00'), Timestamp('2021-02-19 00:00:00'), Timestamp('2021-02-20 00:00:00'), Timestamp('2021-02-21 00:00:00'), Timestamp('2021-02-22 00:00:00'), Timestamp('2021-02-23 00:00:00')]


47953.38
47988.16141460418
 
t0 :
 
prix moyen t0 :  45272.47581038475
 
t0up :  0
prix moyen up t0 :  0.0
 
t0dn :  2
prix moyen dn t0 :  45272.47581038475
 
********
 
t1 :
 
prix moyen t1 :  45888.195083236686
 
t1up :  1
prix moyen up t1 :  47988.16141460418
 
t1dn :  1
prix moyen dn t1 :  43788.228751869196
 


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>